In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from concurrent.futures import ThreadPoolExecutor

import rasterio as rio
from rasterio.crs import CRS

import cv2
from affine import Affine
import numpy as np

from tqdm.notebook import tqdm

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
# REFERENCE IMAGE FROM QGIS
template = Image.open(r"data\GIS\Harris_full_roadsclipped_dark_full.tiff")
# template = Image.open(r"data\GIS\template_cropped.png")

# TILE INDEX TO TEST ON
test_index = Image.open(r"data\000_WorkingFiles\HarrisIndexes\48201CIND0_0992.tif")

In [3]:
def bomb_sides(image, n, replacement_value):
    result = np.copy(image)
    rows, cols = image.shape

    # Replace values in the left and right columns
    result[:, :n] = replacement_value
    result[:, -n:] = replacement_value

    # Replace values in the top and bottom rows
    result[:n, :] = replacement_value
    result[-n:, :] = replacement_value

    return result

def preprocess_image_good(image):
    image = np.asarray(image) * 255
    image = image.astype(np.uint8)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = np.where(image < 150, image, 255).astype(np.uint8)
    image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=1)
    image = cv2.erode(image, np.ones((5,5), np.uint8), iterations=2)
    image = np.where(image < 150, image, 255).astype(np.uint8)
    image = cv2.erode(image, np.ones((5,5), np.uint8), iterations=2)
    image = np.where(image < 50, 0, 255).astype(np.uint8)
    image = cv2.erode(image, np.ones((5,5), np.uint8), iterations=5)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = 255 - image
    return image

def preprocess_image_working(image):
    image = np.asarray(image) * 255
    image = bomb_sides(image, 255, 500)
    image = image.astype(np.uint8)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = cv2.erode(image, np.ones((5,5), np.uint8), iterations=10)
    image = cv2.GaussianBlur(image, (31, 31), 0)
    image = 255 - image
    return image

def preprocess_template(template):
    template = np.asarray(template).astype(np.uint8)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    # template = cv2.GaussianBlur(template, (11, 11), 0)
    # template = 255 * template / np.max(template)
    template = template.astype(np.uint8)
    template = 255 - template
    return template

template = preprocess_template(template)
test_preprocess = preprocess_image_good(test_index)
Image.fromarray(template).save("template.png")
Image.fromarray(test_preprocess).save("test.png")

In [4]:
class ReturnValues:
    def __init__(self, _result, _mask, _rotated_template, _scale, _angle):
        self.result = _result
        self.mask = _mask
        self.rotated_template = _rotated_template
        self.scale = _scale
        self.angle = _angle

def wrapPattern(scale):
    return patternMatch(test_preprocess, template, scale, 0)

def patternMatch(test_preprocess, template, scale, angle, method=cv2.TM_SQDIFF):
    # Resize the template based on the current scale
    scaled_template = cv2.resize(template, None, fx=scale, fy=scale)
    # Rotate the template based on the current angle
    rotation_matrix = cv2.getRotationMatrix2D((scaled_template.shape[1] / 2, scaled_template.shape[0] / 2), angle, 1.0)
    rotated_template = cv2.warpAffine(scaled_template, rotation_matrix, (scaled_template.shape[1], scaled_template.shape[0]))
    mask = np.where(rotated_template == 0, 0, 255).astype(np.uint8)
    # Perform template matching
    result = cv2.matchTemplate(test_preprocess, rotated_template, method, mask=mask)
    return ReturnValues(result, mask,rotated_template, scale, angle)

def postprocess_results(result_list, scales, opt_max):
    elem_list = [x.flatten() for x in result_list]
    elem_list = np.hstack(elem_list)

    if opt_max:
        thresh = np.percentile(elem_list, 99.9)
        loc_list = [x > thresh for x in result_list]
    else:
        thresh = np.percentile(elem_list, 0.1)
        loc_list = [x < thresh for x in result_list]

    votes = np.array([np.count_nonzero(x) for x in loc_list])
    rescale_factor = np.sum(votes * scales) / np.sum(votes)

    x_list = np.hstack([np.where(x)[0] for x in loc_list])
    y_list = np.hstack([np.where(x)[1] for x in loc_list])

    x = int(np.median(x_list))
    y = int(np.median(y_list))

    br = (int(y + template.shape[1] * rescale_factor), int(x + template.shape[0] * rescale_factor))
    
    return x, y, br, rescale_factor

In [12]:
def wrapSearch(initial_guess, perturbance, retall=True):
    scales = np.arange(initial_guess-perturbance, initial_guess+perturbance+1e-5, perturbance/2)

    # Initialize a list to store the matches
    all_matches = []

    # Initialize variables to store the best match
    best_match_scale = 1.0
    best_match_angle = 0.0

    opt_max = False

    if opt_max:
        best_match_value = -1 * np.inf
    else:
        best_match_value = np.inf

    result_list = list()
    best_loc_list = list()
    rect_corner_list = list()

    with ThreadPoolExecutor(max_workers=4) as executor:
        for retvalue in tqdm(executor.map(wrapPattern, scales), total=len(scales)):

            mask = retvalue.mask
            result = retvalue.result
            rotated_template = retvalue.rotated_template
            scale = retvalue.scale
            angle = retvalue.angle
            if not opt_max:
                result = np.sqrt(result / np.count_nonzero(mask))

            result_list.append(result)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if opt_max:
                best_val = max_val
                best_loc = max_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))

                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)

            else:
                best_val = min_val
                best_loc = min_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))
                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)            

            print(scale, angle, best_val)
        
        postprocess = postprocess_results(result_list, scales, opt_max)
    
    if not retall:
        return postprocess[3]
    return postprocess

In [13]:
x, y, br, rescale_factor = wrapSearch(wrapSearch(0.8, 0.1, retall=False), 0.05)

  0%|          | 0/5 [00:00<?, ?it/s]

0.7000000000000001 0 127.24490902172191
0.7500000000000001 0 127.21989386689756
0.8000000000000002 0 127.10752047488349
0.8500000000000002 0 127.04201236836995
0.9000000000000002 0 127.13275329192847


  0%|          | 0/5 [00:00<?, ?it/s]

0.7906438472119354 0 127.14541409241517
0.8156438472119354 0 127.10745711591075
0.8406438472119354 0 127.05490543051535
0.8656438472119354 0 126.91103932303739
0.8906438472119355 0 127.0736473896806


In [14]:
rescale_factor

0.8559098638058965

In [15]:
three_band = np.dstack([test_preprocess,test_preprocess,test_preprocess])
    
# _ = cv2.rectangle(three_band, best_match_loc, best_match_br, (0, 0, 255), 5)
_ = cv2.rectangle(three_band, (y, x), br, (0, 255, 0), 5)

In [16]:
cv2.imwrite('testout.png', three_band)

True